In [1]:
import os

In [2]:
%pwd 

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project'

schema.yaml has the target column, 
our hyoerparameter is stored in params.yaml

## enitity

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_iter: int
    target_column: str

## Config manager

In [6]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_iter = params.max_iter,
            target_column = schema.name
        )

        return model_trainer_config




## Components

In [7]:
import pandas as pd 
import os 
from mlproject import logger 
from sklearn.linear_model import LogisticRegression
import joblib 

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        x_train = train_data.drop([self.config.target_column], axis = 1)
        x_test = test_data.drop([self.config.target_column], axis = 1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lr = LogisticRegression(max_iter= self.config.max_iter, random_state= 2)
        lr.fit(x_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [9]:
try:
    config = ConfigurationManager() # initializing the configuration manager
    model_trainer_config = config.get_model_trainer_config() # getting our model trainer config
    model_trainer_config = ModelTrainer(config=model_trainer_config) # applying the model_trainer_config to our modeltrainer
    model_trainer_config.train() # trainging the model
except Exception as e:
    raise e

[2025-09-09 16:54:35,347: INFO: common: yaml file: config\config.yaml, loaded successfully]
[2025-09-09 16:54:35,354: INFO: common: yaml file: params.yaml, loaded successfully]
[2025-09-09 16:54:35,362: INFO: common: yaml file: schema.yaml, loaded successfully]
[2025-09-09 16:54:35,366: INFO: common: created directory at: artifacts]
[2025-09-09 16:54:35,425: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\ahmed\OneDrive\Documents\Documents\coding\data science\DataScience\MLOPS-Project\venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahmed\OneDrive\Documents\Documents\coding\data science\DataScience\MLOPS-Project\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
